Import data from database

In [7]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# connect to sqlite3 database

# Replace with your database file path in Google Drive
db_path = '/content/drive/MyDrive/House Loan Analytics/my_database.db'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

df= pd.read_sql("SELECT * FROM Property_Values", conn)
# Example query (replace with your actual query)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

conn.close()

[('raw_data',), ('Loan_Approval',), ('Interest_Rate_Features',), ('Risk_classifier',), ('Property_Values',)]


In [4]:
df

,purchaser_type,preapproval,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,loan_term,negative_amortization,...,occupancy_type_3,manufactured_home_secured_property_type_2,manufactured_home_secured_property_type_1111,manufactured_home_land_property_interest_2,manufactured_home_land_property_interest_3,manufactured_home_land_property_interest_4,submission_of_application_2,submission_of_application_1111,initially_payable_to_institution_2,property_value
0,2,1,0,0,0,205000.0,80.0,7.000,360.0,0,...,0,0,0,0,0,0,0,0,0,205000.0
1,2,1,0,0,0,235000.0,80.0,6.500,360.0,0,...,0,0,0,0,0,0,0,0,0,235000.0
2,2,1,0,0,0,505000.0,80.0,5.625,360.0,0,...,0,0,0,0,0,0,0,0,0,505000.0
3,2,1,0,0,0,685000.0,80.0,6.375,360.0,0,...,0,0,0,0,0,0,0,0,0,755000.0
4,2,1,0,0,0,455000.0,80.0,5.500,360.0,0,...,0,0,0,0,0,0,0,0,0,455000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414802,2,1,0,0,0,205000.0,80.0,6.625,360.0,0,...,0,0,0,0,0,0,0,0,0,205000.0
414803,2,1,0,0,0,305000.0,80.0,6.000,360.0,0,...,0,0,0,0,0,0,0,0,0,305000.0
414804,2,1,0,0,0,665000.0,80.0,5.625,360.0,0,...,0,0,0,0,0,0,0,0,0,1165000.0
414805,2,1,0,0,0,325000.0,80.0,6.625,360.0,0,...,0,0,0,0,0,0,0,0,0,335000.0


Model

In [10]:
buffer_percentage = 0.1

target = 'property_value'


X = df.drop(columns=[target])
y = df[target]

# For modeling, convert categorical columns with One-Hot Encoding if present
X_encoded = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Standard regression metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Calculate buffer-based accuracy
    buffer_lower = y_test * (1 - buffer_percentage)
    buffer_upper = y_test * (1 + buffer_percentage)

    within_buffer = np.logical_and(y_pred >= buffer_lower, y_pred <= buffer_upper)
    accuracy_buffer = within_buffer.mean() * 100  # as percentage

    # Print comprehensive performance clearly
    print(f"\n{name} Model Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")
    print(f"Accuracy within ±{buffer_percentage*100:.0f}% buffer: {accuracy_buffer:.2f}%")


Linear Regression Model Performance:
MAE: 172880.33
RMSE: 2272647.64
R²: 0.6242
Accuracy within ±10% buffer: 21.52%

Decision Tree Model Performance:
MAE: 102927.67
RMSE: 2455328.74
R²: 0.5614
Accuracy within ±10% buffer: 69.68%

Random Forest Model Performance:
MAE: 78771.59
RMSE: 2178550.79
R²: 0.6547
Accuracy within ±10% buffer: 73.24%

XGBoost Model Performance:
MAE: 156821.47
RMSE: 7480067.83
R²: -3.0707
Accuracy within ±10% buffer: 60.02%
